In [5]:
import json
def read_schemas():
    # create decision table
    with open("eventnative/schemas/decisions.json") as decisions_schema:
        fields = json.load(decisions_schema)
        print(fields)
        #create_table(client, args.dataset, "decisions", fields)

    # create rewards table
    with open("eventnative/schemas/rewards.json") as rewards_schema:
        fields = json.load(rewards_schema)
        #create_table(client, args.dataset, "rewards", fields)
        print(fields)

    print("ML bandit dataset and tables created.")
    
    
read_schemas()

[{'mode': 'REQUIRED', 'name': 'decision_id', 'type': 'STRING'}, {'mode': 'REQUIRED', 'name': 'context', 'type': 'STRING'}, {'mode': 'REQUIRED', 'name': 'decision', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'experiment_id', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'variation_id', 'type': 'INTEGER'}, {'mode': 'NULLABLE', 'name': 'mdp_id', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'ts', 'type': 'INTEGER'}, {'mode': 'NULLABLE', 'name': 'score', 'type': 'FLOAT'}]
[{'mode': 'NULLABLE', 'name': 'decision_id', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'decision', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'metrics', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'experiment_id', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'mdp_id', 'type': 'STRING'}, {'mode': 'NULLABLE', 'name': 'ts', 'type': 'INTEGER'}]
ML bandit dataset and tables created.


In [6]:
CURRENT_YEAR = 2020
YEARLY_MEAN_CM_ADJUSTMENTS = 0.25
POSSIBLE_YEARS = range(CURRENT_YEAR - 50, CURRENT_YEAR + 1)

NUM_GET_DECISION_CALLS = 10000
POSSIBLE_YEARS

range(1970, 2021)

In [7]:
import modelcloud

-- app dirs (with optional 'version')


In [10]:
import modelcloud
from modelcloud.data_types import text,number,any,category


In [7]:
%%file decisionLogServer.py
import modelcloud
from modelcloud.data_types import text,number,any,category

from eventnative.bandit_data_model import Bandit_decisions_model,Bandit_rewards_model
from peewee import *
d = "/Users/leepand/Downloads/github/vue/vue-admin-flask-example/algolink_ui_server/files/localdb/localdb3-v1/model_samples.db"
db = SqliteDatabase(d)

decision_inputs = {
    'dataset_id': text(description="数据ID"),
    'project_id': text(description="项目ID"),
    'experiment_id': text(description="实验ID"),
    'mdp_id': text(description="马尔科夫决策ID"),
    'decision_id': text(description="推荐决策ID"),
    'decision': text(description="决策内容"),
    'ts': number(description="时间戳"),
    'variation_id': text(description="实验用例"),
    'context': text(description="特征内容"),
    'score' :any(description="决策的预期得分")
}
decisions_outputs = {
    'result': text(description="数据入库标识")
}

decision_description = '记录推荐系统的决策信息'

@modelcloud.command('logDecision', 
                    inputs=decision_inputs, 
                    outputs=decisions_outputs,
                    description=decision_description)
def decision_model(model, decisions):
    prefix= decisions["dataset_id"]
    project_id= decisions["project_id"]
    decision_id = decisions["decision_id"]
    DecisionModel = Bandit_decisions_model(prefix,project_id,db)
    DecisionModel(decision_id =decision_id,
                  context = decisions["context"],
                  experiment_id = decisions["experiment_id"],
                  decision = decisions["decision"],
                  mdp_id = decisions["mdp_id"],
                  variation_id = decisions["variation_id"],
                  ts = decisions["ts"],
                  score = decisions["score"]
                 ).save()
    return {"result":"True"}
    
reward_inputs = {
    'dataset_id': text(description="数据ID"),
    'project_id': text(description="项目ID"),
    'experiment_id': text(description="实验ID"),
    'mdp_id': text(description="马尔科夫决策ID"),
    'decision_id': text(description="推荐决策ID"),
    'decision': text(description="决策内容"),
    'ts': number(description="时间戳"),
    'metrics': text(description="奖励指标的映射")
}
reward_outputs = {
    'result': text(description="数据入库标识")
}

reward_description = '记录推荐系统的用户反馈信息'

@modelcloud.command('logReward', 
                    inputs=reward_inputs, 
                    outputs=reward_outputs,
                    description=reward_description)
def decision_model(model, rewards):
    prefix= rewards["dataset_id"]
    project_id= rewards["project_id"]
    decision_id = rewards["decision_id"]
    RewardModel = Bandit_rewards_model(prefix,project_id,db)
    if decision_id=="null":
        RewardModel(metrics = rewards["metrics"],
                    experiment_id = rewards["experiment_id"],
                    mdp_id = rewards["mdp_id"],
                    ts = rewards["ts"]
                   ).save()
    else:
        RewardModel(decision_id = rewards["decision_id"],
                    decision = rewards["decision"],
                    metrics = rewards["metrics"],
                    experiment_id = rewards["experiment_id"],
                    mdp_id = rewards["mdp_id"],
                    ts = rewards["ts"]
                   ).save()
    return {"result":"True"}

if __name__ == '__main__':
    modelcloud.run(port=9003,worker_class="gevent",runenv="dev")

Writing decisionLogServer.py


In [16]:
setup_options={
  'architecture': category(choices=['Artistic', 'Stable', 'Video'])
}
setup_options['architecture'].__dict__

{'type': 'category',
 'description': None,
 'name': 'category',
 'choices': ['Artistic', 'Stable', 'Video'],
 'default': 'Artistic'}

In [ ]:
                    <router-link :to="{ name: '模型列表', params: {project_id: this.$route.params.project_id}}" style='text-decoration: none;color: aliceblue;'>
                        所有模型
                    </router-link>

In [ ]:
                    <router-link :to="{ name: '分组模型列表', params: {project_id: this.$route.params.project_id,firstGroup: firstID}}" style='text-decoration: none;color: aliceblue;'>
                        所有模型
                    </router-link>

In [2]:
from websocket import create_connection

ws = create_connection('ws://localhost:9002/')

In [17]:
import json
def create_ws_message(message_type, data):
    return json.dumps(dict(type=message_type, **data))
ws.send(create_ws_message('submit', dict(command='fib', id='test9', inputData={'n': 10})))

response = json.loads(ws.recv())
response

{'type': 'started', 'id': 'test9'}

In [18]:
response = json.loads(ws.recv())
response

{'type': 'output', 'id': 'test9', 'outputData': {'result': 89}}

In [9]:
response

{'type': 'output', 'id': 'test7', 'outputData': {'result': 89}}

In [3]:
import platform
def get_system_info():
    system = platform.system()

    if system == "Windows":
        version = sys.getwindowsversion()

        return {
            "os": "windows",
            "windows_version_build": version.build,
            "windows_version_major": version.major,
            "windows_version_minor": version.minor,
            "windows_version_service_pack": version.service_pack,
        }

    if system == "Darwin":
        return {"os": "mac", "mac_version": platform.mac_ver()[0]}

    if system == "Linux":
        return {
            "os": "linux",
            "linux_distro": distro.id(),
            "linux_distro_like": distro.like(),
            "linux_distro_version": distro.version(),
        }

    # We don't collect data for any other system.
    return {"os": "unknown"}

get_system_info()

{'os': 'mac', 'mac_version': '10.16'}

In [2]:
get_system_info()

{'os': 'mac', 'mac_version': '10.16'}